In [134]:
import pandas as pd
import numpy as np
import math
import glob
import os

In [135]:
wd = os.getcwd()
files = glob.glob(wd + '/*.xlsx')
for f in files:
    if "US Open" in f:
        files.remove(f)

In [136]:
def probability(elo_1, elo_2):
    
    return 1/(1 + math.pow(10, (elo_1 - elo_2)/400))

In [137]:
def elo_rating(rating_player, rating_opponent, K):
    
    prob_a = probability(rating_opponent, rating_player)
    prob_b = probability(rating_player, rating_opponent)
    
    rating_player = rating_player + K * (1 - prob_a)
    rating_opponent = rating_opponent + K * (0 - prob_b)
    
    return rating_player, rating_opponent

In [141]:
def apply_elo(df):

    for i in range(df.shape[0]):
        elo_winner, elo_loser = elo_rating(df.at[i, 'winner_elo'], df.at[i, 'loser_elo'], 32)

        df.at[i, 'winner_elo'] = np.around(elo_winner)
        df.at[i, 'loser_elo'] = np.around(elo_loser)

        winner_name = df.winner_name[i]
        loser_name = df.loser_name[i]

        sub_ww = df.iloc[i:, :][df.iloc[i:, :]['winner_name']==winner_name].winner_elo
        sub_ll = df.iloc[i:, :][df.iloc[i:, :]['loser_name']==loser_name].loser_elo
        sub_lw = df.iloc[i:, :][df.iloc[i:, :]['loser_name']==winner_name].loser_elo
        sub_wl = df.iloc[i:, :][df.iloc[i:, :]['winner_name']==loser_name].winner_elo
        
        sub_ww.replace(to_replace=sub_ww.values.tolist(), value=np.around(elo_winner), inplace=True)
        sub_ll.replace(to_replace=sub_ll.values.tolist(), value=np.around(elo_loser), inplace=True)
        sub_lw.replace(to_replace=sub_lw.values.tolist(), value=np.around(elo_winner), inplace=True)
        sub_wl.replace(to_replace=sub_wl.values.tolist(), value=np.around(elo_loser), inplace=True)
        
        df.update(sub_ww)
        df.update(sub_ll)
        df.update(sub_lw)
        df.update(sub_wl)
        
    return df

In [173]:
def season_final_elo(df):
    
    list1 = df.winner_name.unique()
    list2 = df.loser_name.unique()
    names = np.unique(np.concatenate((list1, list2))).tolist()
    
    my_list = []
    for name in names:
        sub_df = df[((df['winner_name']==name) | (df['loser_name']==name))]
        last = sub_df.iloc[-1,:]
        
        if last.winner_name != name:
            case = {'Name': name, 'elo': last.loser_elo}
            my_list.append(case)
        else:
            case = {'Name': name, 'elo': last.winner_elo}
            my_list.append(case)
            
    return my_list, names

In [ ]:
sub_df = df[((df['winner_name']=='Marcos Baghdatis') | (df['loser_name']=='Marcos Baghdatis'))]
    last = sub_df.iloc[-1,:]

In [139]:
df0 = pd.read_excel(files[0])
df0.tourney_date = pd.to_datetime(df0.tourney_date, yearfirst=True, format='%Y%m%d')
df0 = df0.sort_values(by='tourney_date').reset_index(drop=True)

data = np.ones((len(df0), 1))*1500
df0['winner_elo'] = data
df0['loser_elo'] = data

df0 = apply_elo(df0)

In [164]:
df0[['winner_name', 'loser_name', 'winner_elo', 'loser_elo']].tail(51)

,winner_name,loser_name,winner_elo,loser_elo
2628,Fernando Verdasco,Arnaud Clement,1620.0,1427.0
2629,Gilles Simon,Andrey Golubev,1642.0,1540.0
2630,Robin Soderling,Michael Llodra,1750.0,1543.0
2631,Robin Soderling,Gael Monfils,1767.0,1753.0
2632,Robin Soderling,Andy Roddick,1782.0,1726.0
2633,Robin Soderling,Gilles Simon,1792.0,1632.0
2634,Robin Soderling,Stan Wawrinka,1803.0,1668.0
2635,Sergiy Stakhovsky,Illya Marchenko,1531.0,1473.0
2636,Radek Stepanek,Nicolas Almagro,1560.0,1608.0
2637,Radek Stepanek,Michael Russell,1569.0,1401.0


In [174]:
my_list, names = season_final_elo(df0)

In [185]:
for name in names:

    sub_df_win = df0[df0['winner_name']==name].winner_elo
    sub_df_loss = df0[df0['loser_name']==name].loser_elo
print(sub_df_win)    

Series([], Name: winner_elo, dtype: float64)


In [161]:
my_dict

{'Name': ['Marcos Baghdatis'], 'elo': [1593.0]}

In [156]:
list1 = df0.winner_name.unique()
list2 = df0.loser_name.unique()

#new_df = pd.DataFrame()

dd = np.concatenate((list1, list2))
np.unique(dd).tolist()

['Abdulla Hajji',
 'Adrian Mannarino',
 'Adrian Ungur',
 'Agustin Velotti',
 'Albert Montanes',
 'Albert Ramos-Vinolas',
 'Alberto Martin',
 'Alejandro Falla',
 'Alessio Di Mauro',
 'Alex Bogdanovic',
 'Alex Bogomolov',
 'Alexander Kudryavtsev',
 'Alexander Peya',
 'Alexander Sadecky',
 'Alexandr Dolgopolov',
 'Alexandre Sidorenko',
 'Andreas Beck',
 'Andreas Haider-Maurer',
 'Andreas Seppi',
 'Andreas Vinciguerra',
 'Andrej Martin',
 'Andrey Golubev',
 'Andrey Kuznetsov',
 'Andy Murray',
 'Andy Roddick',
 'Antonio Veic',
 'Arnaud Clement',
 'Bastian Knittel',
 'Benjamin Balleret',
 'Benjamin Becker',
 'Benoit Paire',
 'Bernard Tomic',
 'Bjorn Phau',
 'Blaz Kavcic',
 'Bobby Reynolds',
 'Bradley Klahn',
 'Brendan Evans',
 'Brian Dabul',
 'Carlos Berlocq',
 'Carlos Moya',
 'Carsten Ball',
 'Chris Guccione',
 'Christian Lindell',
 'Christophe Rochus',
 'Conor Niland',
 'Cristobal Saavedra Corvalan',
 'Danai Udomchoke',
 'Daniel Brands',
 'Daniel Evans',
 'Daniel Gimeno-Traver',
 'Daniel K

In [39]:
df1 = pd.read_excel(files[0])
df2 = pd.read_excel(files[1])

df1_cols = df1.columns
df2_cols = df2.columns

In [42]:
len(df2_cols)

78

In [38]:
df1.equals(df2)

False

In [46]:
init_elo = 1500

for i in files:

    df = pd.read_excel(i)
    df.tourney_date = pd.to_datetime(df.tourney_date, yearfirst=True, format='%Y%m%d')
    df = df.sort_values(by='tourney_date').reset_index(drop=True)
    
    data = np.ones((len(df), 1))*init_elo
    df['winner_elo'] = data
    df['loser_elo'] = data


C:\Users\PC\Desktop\Hasl\Rahber Tariani\2010_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2011_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2012_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2013_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2014_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2015_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2016_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2017_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2018_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2019_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2020_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2021_mens.xlsx


In [313]:
df.tourney_date = pd.to_datetime(df.tourney_date, yearfirst=True, format='%Y%m%d')
df = df.sort_values(by='tourney_date').reset_index(drop=True)

In [314]:
df.shape

(2679, 78)

In [315]:
df

,tourney_id,tourney_name,series,court,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,...,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,6,104571,...,1.40,2.75,1.377,3.410,1.400,2.750,NaN,NaN,NaN,NaN
1,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,30,103285,...,2.10,1.67,2.350,1.685,2.000,1.727,NaN,NaN,NaN,NaN
2,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,16,103285,...,1.04,9.00,1.087,9.950,1.030,10.000,NaN,NaN,NaN,NaN
3,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,24,103285,...,1.12,5.50,1.159,6.390,1.125,5.500,NaN,NaN,NaN,NaN
4,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,11,103722,...,1.44,2.62,1.483,2.900,1.440,2.630,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,1,103819,...,1.13,5.50,1.140,7.100,1.130,6.000,1.14,8.20,1.11,6.39
2675,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,4,103819,...,1.44,2.63,1.420,3.180,1.360,3.000,1.50,3.24,1.40,2.92
2676,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,14,104925,...,1.36,3.00,1.480,2.920,1.440,2.750,1.48,3.15,1.40,2.88
2677,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,13,104925,...,1.25,3.75,1.250,4.600,1.250,3.750,1.30,4.76,1.24,4.03
